# OpenIntro Statistics - Chapter 1 : Introduction to Data (Introduction aux données)
## Titanic Survival Analysis (Titanic dataset)
>**Source**: *OpenIntro Statistics*(4th ed.)<br>
>**Core Principle**:*'Variable type determines analysis method'*<br>
[OpenIntro Ch 1 Theory summary](../references/openintro_ch1_summary_md.md)


In [ ]:
# Data loading and Initial Inspection.
# • Observation = line/ligne/行 (如: 一名乘客--un case)
# • Variable = column/列 (如: age, sex)
# • Categorical: nominal(sex/性别) / ordinal(pclass/舱位)
# • Numerical: discrete(sibsp--only integer-0,4,2) / continuous(age--float--1.5)
# • Response Y = survived | Explanatory X = pclass, sex, age
# Observational studies → Association ≠ Causation (cannot prove that "buying first class resulted in survival"

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Load built-in Titanic dataset
df = sb.load_dataset('titanic')

print("*"*60)
print("step 1.Check dataset") 
print("*"*60)
print(f"Dimension of titanic dataset: {df.ndim}") # savoi
print(f"Observations of titanic dataset: {df.shape[0]:,} passengrs")
print(f"Variables of titanic dataset: {df.shape[1]:} features")
print("List variables group by data type")
#convert all the type (categorial) to str for group by.
dtype_counts = df.dtypes.astype(str).value_counts()
#print(f"dtype_counts\n{dtype_counts}")
for dtype, count in dtype_counts.items():
    #create list for datatype and columns.
    cols = df.select_dtypes(
        include=[dtype]
    ).columns.tolist()
    #print(f"hello{cols}")
    print(f"  • {dtype:15s}: {count:2d} columns ({', '.join(cols)})")
print(f"Show the first three lines")
display(df.head(3))